# $ Tidy Data $

#### Imports

In [3]:
import pandas as pd
import env
from pydataset import data
import acquire

# train test split from sklearn
from sklearn.model_selection import train_test_split
# imputer from sklearn
# help with missing value by replacing blank with: median, mode, average, calculate using other column
from sklearn.impute import SimpleImputer

db_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

### 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:


name

Billy    0.5250 

Jane     0.6875

John     0.9125

Sally    0.7625

Name: grade, dtype: float64
Coffee Levels

In [ ]:
db_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

   

sql_query = """
                SELECT * 
                From attendance;
                """

attendance = pd.read_sql(sql_query, db_url)

In [ ]:
# check to see what attendance looks like
attendance

In [ ]:
# create a dataframe where we melt data and attendance
df = attendance.melt(id_vars='Unnamed: 0',var_name= 'date', value_name='attendance')
df

In [ ]:
# rename columns to name
df.rename(columns={'Unnamed: 0':'name'}, inplace = True)

# drop date column
df.drop(columns ='date', inplace = True)

In [ ]:
# replace data values of attendance with floats relating to attendance
df['attendance'].replace({'A':0,'P':1,'T':.9,'H':0.5}, inplace = True)

In [ ]:
# group by name to obtain average attendance
attendance_avg = df.groupby('name').mean().astype('float')

In [ ]:
attendance_avg

### 2. Coffee Levels

### a) Read the coffee_levels table.

In [21]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From coffee_levels;
                """

coffee_levels = pd.read_sql(sql_query, db_url )

In [22]:
coffee_levels


,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


### b) Transform the data so that each carafe is in it's own column.

In [24]:
# Transform the data so that each carafe is its own columns
coffee_levels.pivot_table(index=['hour'],
                         columns = 'coffee_carafe',
                         values = 'coffee_amount').reset_index()

coffee_carafe,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


### c) Is this the best shape for the data?

The shape above is a better shape for the table because coffee_carafe is best seen as a column.

### 3. Cake Recipes

### a) Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [ ]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From cake_recipes;
                """

cake_recipes = pd.read_sql(sql_query, db_url)

In [ ]:
# see the data
cake_recipes

### b) Tidy the data as necessary.

In [ ]:
# split recipe and position to create individual observations, set expand = True so each variable gets its own columns
recipe_position_df = cake_recipes['recipe:position'].str.split(':', expand= True)
recipe_position_df.columns =['recipe', 'position']
recipe_position_df.head()

In [ ]:
# dropped concat original dataframe with split variable dataframe and drop column
cake2=pd.concat([recipe_position_df,cake_recipes], axis = 1).drop(columns = 'recipe:position')
cake2.head()

In [ ]:
# create a melt and keep vairables as columns ['recipe','position'] and convert the rest to 'temp'column, use values as rate values
cake_melt = cake2.melt(id_vars=['recipe','position'],
                 var_name='temp',
                 value_name='rate')

In [ ]:
# tidy data
cake_melt

### c) Which recipe, on average, is the best? 
Recipe b has the highest mean 76.73607

In [ ]:
# group by recipe type and calculate the mean
cake_melt.groupby('recipe').mean()

### d) Which oven temperature, on average, produces the best results? 
The oven temperature of 275 producees the best results according to the rating of 74.886754


In [ ]:
cake_melt.groupby('temp').mean()

### e) Which combination of recipe, rack position, and temperature gives the best result?
Recipe B in bottom position at 300 degrees has the highest rating at 99.248541

In [ ]:
cake_melt[cake_melt.rate == cake_melt.rate.max()]

# 4 Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.

In [4]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From gapminder1;
                """

gapminder1 = pd.read_sql(sql_query, db_url)

In [14]:
gapminder1.sample(10)

,year,country,measure,measurement
225,1980,Egypt,pop,4.263422e+07
1531,1965,China,fertility,6.060000e+00
1466,1970,Barbados,fertility,2.740000e+00
1316,1990,South Africa,life_expect,6.188800e+01
1408,1955,Aruba,fertility,5.150000e+00
1697,1970,Haiti,fertility,5.600500e+00
816,1965,Canada,life_expect,7.213000e+01
246,1975,Finland,pop,4.711439e+06
641,1970,Switzerland,pop,6.267000e+06
1881,1955,Netherlands,fertility,3.095000e+00


 Measure pop, fertility, life_expect look like variables with measurement as values,
 So we need to go up ...pivot table

In [9]:
gapminder1.columns

Index(['year', 'country', 'measure', 'measurement'], dtype='object')

In [12]:
gapminder1.dtypes

year             int64
country         object
measure         object
measurement    float64
dtype: object

In [20]:
gapminder1.pivot_table(index=['year','country'],
                        columns = 'measure',
                      values = 'measurement').reset_index()

measure,year,country,fertility,life_expect,pop
0,1955,Afghanistan,7.7000,30.332,8891209.0
1,1955,Argentina,3.1265,64.399,18927821.0
2,1955,Aruba,5.1500,64.381,53865.0
3,1955,Australia,3.4060,70.330,9277087.0
4,1955,Austria,2.5200,67.480,6946885.0
...,...,...,...,...,...
688,2005,Switzerland,1.4200,81.701,7489370.0
689,2005,Turkey,2.1430,71.777,69660559.0
690,2005,United Kingdom,1.8150,79.425,60441457.0
691,2005,United States,2.0540,78.242,295734134.0
